<a href="https://colab.research.google.com/github/kch1103/my/blob/main/svhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch 
import torchvision
import torchvision.transforms as transforms 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 
from sklearn.preprocessing import LabelBinarizer
from torchvision import models
from torchsummary import summary
from torchvision.datasets import SVHN

In [15]:
resnet18_pretrained = models.resnet18(pretrained = True)
print(resnet18_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
svhn_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [17]:
train_set = SVHN(
    root = './data',
    split = 'train',
    download = True,
    transform = svhn_transform
)

Using downloaded and verified file: ./data/train_32x32.mat


In [18]:
test_set = SVHN(
    root = './data',
    split = 'test',
    download = True,
    transform = svhn_transform
)

Using downloaded and verified file: ./data/test_32x32.mat


In [19]:
train_data = torch.utils.data.DataLoader(dataset = train_set,
                                         batch_size = 10,
                                         shuffle = True)

test_data = torch.utils.data.DataLoader(dataset = test_set,
                                        batch_size = 10,
                                        shuffle = True) 



In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [21]:
num_classes = 10
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

resnet18_pretrained.to(device)
model = resnet18_pretrained

In [22]:
learning_rate = 0.001
epochs = 2

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_data, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')


[1,    10] loss: 3.301
[1,    20] loss: 2.722
[1,    30] loss: 3.025
[1,    40] loss: 2.615
[1,    50] loss: 2.683
[1,    60] loss: 2.194
[1,    70] loss: 2.533
[1,    80] loss: 2.248
[1,    90] loss: 2.161
[1,   100] loss: 2.357
[1,   110] loss: 2.264
[1,   120] loss: 2.133
[1,   130] loss: 2.092
[1,   140] loss: 2.241
[1,   150] loss: 2.146
[1,   160] loss: 2.142
[1,   170] loss: 2.037
[1,   180] loss: 2.198
[1,   190] loss: 2.038
[1,   200] loss: 2.191
[1,   210] loss: 2.091
[1,   220] loss: 2.059
[1,   230] loss: 2.389
[1,   240] loss: 2.051
[1,   250] loss: 2.111
[1,   260] loss: 2.109
[1,   270] loss: 2.088
[1,   280] loss: 2.177
[1,   290] loss: 2.073
[1,   300] loss: 1.992
[1,   310] loss: 2.119
[1,   320] loss: 1.841
[1,   330] loss: 2.016
[1,   340] loss: 2.049
[1,   350] loss: 2.028
[1,   360] loss: 2.066
[1,   370] loss: 1.729
[1,   380] loss: 1.562
[1,   390] loss: 1.616
[1,   400] loss: 1.889
[1,   410] loss: 1.815
[1,   420] loss: 1.644
[1,   430] loss: 1.741
[1,   440] 

In [27]:
classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
  for data in test_data:
    images, labels = data 
    images = images.cuda()
    labels = labels.cuda()

    outputs = model(images)
    _, predictions = torch.max(outputs, 1)

    for label, prediction in zip(labels, predictions):
      if label == prediction:
        correct_pred[classes[label]] += 1
      total_pred[classes[label]] += 1

In [28]:
for classname, correct_count in correct_pred.items():
  accuracy = 100 * float(correct_count) / total_pred[classname]
  print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))

Accuracy for class 0     is: 87.7 %
Accuracy for class 1     is: 91.4 %
Accuracy for class 2     is: 91.9 %
Accuracy for class 3     is: 86.1 %
Accuracy for class 4     is: 90.6 %
Accuracy for class 5     is: 90.8 %
Accuracy for class 6     is: 85.5 %
Accuracy for class 7     is: 93.4 %
Accuracy for class 8     is: 79.0 %
Accuracy for class 9     is: 91.3 %
